# LLM with Web Search and Crawl

Code to crawl the top n pages of a Google search result and serve them to LLM in order to utilize rich context.



In [16]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True) 

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

bs4 or scrapy?

In [17]:
from typing import List, Tuple
import requests
import json
import scrapy
from bs4 import BeautifulSoup
import httpx
import asyncio
from urllib.parse import urljoin
from azure.ai.projects.models import MessageRole, BingGroundingTool
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
BING_GROUNDING_PROJECT_CONNECTION_STRING = os.getenv("BING_GROUNDING_PROJECT_CONNECTION_STRING")
BING_GROUNDING_AGENT_ID = os.getenv("BING_GROUNDING_AGENT_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_CONNECTION_NAME = os.getenv("BING_GROUNDING_CONNECTION_NAME")
# Web search mode: "google" or "bing"
# it can be changed when users want to use different search engine
WEB_SEARCH_MODE = os.getenv("WEB_SEARCH_MODE")
WEB_CRAWLING_MODE = os.getenv("WEB_CRAWLING_MODE") #on or off

def extract_text_and_tables_by_bs4(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Extract main text
    paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
    text = "\n".join(paragraphs)
    return text



async def extract_contexts_async(url_snippet_tuples: List[Tuple[str, str]]) -> List[str]:
    """
    Asynchronously extract content from a list of URLs with their snippets.
    
    Args:
        url_snippet_tuples: List of (url, snippet) pairs to process
        
    Returns:
        List of extracted contents
    """
    async def fetch(url: str, snippet: str) -> str:
        # Try to get from cache first
        
        # If not in cache or cache unavailable, fetch the content
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        }
        
        try:
            async with httpx.AsyncClient(timeout=10, follow_redirects=True) as client:
                try:
                    response = await client.get(url, headers=headers)
                    response.raise_for_status()
                except httpx.HTTPStatusError as e:
                    # Handle redirects manually if needed
                    if e.response.status_code == 302 and "location" in e.response.headers:
                        redirect_url = e.response.headers["location"]
                        if not redirect_url.startswith("http"):
                            redirect_url = urljoin(url, redirect_url)
                        try:
                            response = await client.get(redirect_url, headers=headers)
                            response.raise_for_status()
                        except Exception as e2:
                            print(f"Redirect request failed: {e2}")
                            return f"{snippet} "
                    else:
                        print(f"Request failed: {e}")
                        return f"{snippet} "
                except httpx.HTTPError as e:
                    print(f"Request failed: {e}")
                    return f"{snippet} "
                
                # Parse the content
                selector = scrapy.Selector(text=response.text)
                
                # Extract paragraphs
                paragraphs = [p.strip() for p in selector.css('p::text, p *::text').getall() if p.strip()]
                
                # Remove duplicate and very short paragraphs
                filtered_paragraphs = []
                seen_content = set()
                for p in paragraphs:
                    # Skip very short paragraphs that are likely UI elements
                    if len(p) < 5:
                        continue
                    # Avoid duplicate content
                    if p in seen_content:
                        continue
                    seen_content.add(p)
                    filtered_paragraphs.append(p)
                
                # Join the filtered paragraphs
                text = "\n".join(filtered_paragraphs)
                
                # If no paragraphs were found, try to get other text content
                if not text:
                    content_texts = [t.strip() for t in selector.css(
                        'article::text, article *::text, .content::text, .content *::text, '
                        'main::text, main *::text'
                    ).getall() if t.strip()]
                    
                    if content_texts:
                        text = "\n".join(content_texts)
                
                # Combine snippet with extracted text
                snippet_text = f"{snippet}: {text}"
                
                
                return snippet_text
                
        except Exception as e:
            print(f"Error processing URL {url}: {str(e)}")
            return f"{snippet} [Error: {str(e)}]"
    
    # Create tasks for all URLs
    tasks = [asyncio.create_task(fetch(url, snippet)) 
            for url, snippet in url_snippet_tuples]
    
    # Execute all tasks concurrently
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Process results
    processed_results = []
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"Error processing URL {url_snippet_tuples[i][0]}: {str(result)}")
            processed_results.append(f"{url_snippet_tuples[i][1]} [Processing Error]")
        else:
            processed_results.append({"content": result, "url_citation" :{"link": url_snippet_tuples[i][0], "title": url_snippet_tuples[i][1]}})

    return processed_results
                    
           
def google_search(query, num=5, search_type="web"):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query + " -filetype:pdf",
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num, 
        "locale": "ko",  # 한국어로 검색
        "filter": "1"
    }
    
    if search_type == "image":
        params["searchType"] = "image"
        
    response = requests.get(url, params=params)
    results = response.json()
    return results.get("items", [])

def bing_grounding_search(query, num=5, search_type="web"):
    try:
        creds = DefaultAzureCredential()
        
        project_client = AIProjectClient.from_connection_string(
            credential=creds,
            conn_str=BING_GROUNDING_PROJECT_CONNECTION_STRING,
        )
        
        agent_id = BING_GROUNDING_AGENT_ID
        
        if not agent_id:
            print("BING_GROUNDING_AGENT_ID is not set. Create new agent...")
            connection_name = BING_GROUNDING_CONNECTION_NAME
            
            bing_connection = project_client.connections.get(
                connection_name=connection_name,
            )
            conn_id = bing_connection.id
            
            bing = BingGroundingTool(connection_id=conn_id)
            
            
            agent = project_client.agents.create_agent(
                model=BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME,
                name="temporary-bing-agent",
                instructions="""
                    Search for product information and images exclusively about Samsung products. Get all contents from the website as much as you can. Don't include the url link in the response.
                    Only respond with information from trusted sources: samsung.com.
                    Prioritize data from samsung.com whenever available to ensure accuracy and reliability.
                    If information is not found on samsung.com, supplement with tistory.com, but always indicate the source.
                    Avoid using data from any other websites or unverified sources.
                """,
                tools=bing.definitions,
                headers={"x-ms-enable-preview": "true"}
            )
            agent_id = agent.id
            print(f"New agent created. Agent ID: {agent_id}")
        else:
            print(f"Existing agent ID: {agent_id}")
            try:
                agent = project_client.agents.get_agent(agent_id)
            except Exception as agent_error:
                print(f"Failed to retrieve agent: {agent_error}")
                return []

        thread = project_client.agents.create_thread()
        
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=f"Search the web for: {query}. Return only the top {num} most relevant results as a list.",
        )

        
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        
        if run.status == "failed":
            print(f"Execution failed: {run.last_error}")
            return []
        
        results = []
        response_message = project_client.agents.list_messages(thread_id=thread.id).get_last_message_by_role(
            MessageRole.AGENT
        )
        # Extract content text and annotations
        if response_message.content:
            for content_item in response_message["content"]:
                if content_item["type"] == "text":
                    text_content = content_item["text"]["value"]
                    results.append({"content": text_content})
            
        if response_message.url_citation_annotations:
            for annotation in response_message.url_citation_annotations:
                if annotation["type"] == "url_citation":
                    url_citation = annotation["url_citation"]
                    url = url_citation["url"]
                    title = url_citation["title"]
                    results.append({"url_citation":{"link": url, "title": title}})

        if not BING_GROUNDING_AGENT_ID and hasattr(agent, 'id'):
            try:
                print(f"Deleting temporary agent with ID: {agent.id}")
                project_client.agents.delete_agent(agent.id)
                
            except Exception as delete_error:
                print(f"Error deleting agent: {delete_error}")

        return results if results else []
    except Exception as e:
        print(f"Bing Grounding error : {e}")
        return []

def web_search(query, num=5, search_type="web"):
    """환경 변수에 따라 Google Search API 또는 Bing Grounding을 사용하여 검색 수행"""
    
    if WEB_SEARCH_MODE == "bing":
        try:
            return bing_grounding_search(query['llm_query'], num, search_type)
            
        except Exception as e:
            print(f"Bing Grounding 검색 중 오류 발생: {e}")
    else:
        return google_search(query['web_search'], num, search_type)

       
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 구글 검색과 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해.

            1. Web Search용 Query Rewrite:
            - 사용자의 질문을 실제 검색 엔진 검색창에 입력할 수 있도록, 명확하고 간결한 핵심 키워드 중심의 검색어로 재작성해.
            - 불필요한 문장, 맥락 설명은 빼고, 검색에 최적화된 형태로 만들어.
            - 핵심 키워드를 반복적으로 사용해 검색의 정확도를 높여.

            2. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
            * 웹 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해.
            {"web_search": "웹 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())


In [18]:
from IPython.display import Markdown, display
from datetime import datetime
import time

#TODO 날씨나 뉴스, 기타 다른 특정정보는 Function Call
# inputs = ["날씨, 뉴스"] ##

async def process_web_search_call(RESULTS_COUNT, input):
    
    start_time = time.time()
    
    contexts = [] 
    print(f"Original Input: {input}")
    
    query_rewrite = rewrite_query_for_search_and_llm(input, client)
    print(f"Web Search Query: {query_rewrite['web_search']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    results = web_search(query_rewrite, RESULTS_COUNT)
    print(f"Web Search Results: {len(results)}")
    if WEB_SEARCH_MODE == "bing" and results and isinstance(results, list) and len(results) > 0:
        contexts = [results[i] for i in range(len(results))]
        
    elif WEB_SEARCH_MODE == "google" and results and isinstance(results, list) and len(results) > 0:
        url_snippet_tuples = [(r["link"], r["snippet"]) for r in results]
        contexts = await extract_contexts_async(url_snippet_tuples)
        
    else:
        print("No results found or invalid response from web_search.")
        contexts = [] 
    
    # for i, context in enumerate(contexts):
    #     print(f"Context {i+1}: {context}...")  # Print first 1000 chars of each context
    #     print("\n--- End of Context ---\n")

    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    
    
    system_prompt = """
        너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 
        답변은 마크다운으로 이모지를 1~2개 포함해서 작성해줘. 
        contexts를 최대한 활용하여 풍부하게 답변을 해야해. 
        사용자가 질문한 내용에 대해 정확하고 유용한 정보를 제공해야 해. contexts가 부족하면 최소한의 안내만 해줘. 
        url_citation은 사용자가 클릭할 수 있도록 링크를 제공해줘.
        
    """
    user_prompt = f"""
        너는 아래 제공하는 웹검색에서 검색한 contexts를 바탕으로 질문에 대한 답변을 제공해야 해. 
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.
        웹검색에서 제공한 contexts: {contexts}
        질문: {query_rewrite['llm_query']}
        """
    
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    display(Markdown(response.choices[0].message.content))
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.2f} seconds")


In [14]:

RESULTS_COUNT = 3

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    # "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    # "삼성전자 25년 제품이 작년 대비 좋아진것은",
    # "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    # "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    # "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]


WEB_SEARCH_MODE = "google"  

for input in inputs:
    
    print(f"Google Search API 사용: {input}")
    await process_web_search_call(RESULTS_COUNT, input)

WEB_SEARCH_MODE = "bing"

for input in inputs:
    print(f"Bing Grounding 검색 사용: {input}")
    await process_web_search_call(RESULTS_COUNT, input)    

Google Search API 사용: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘


Web Search Query: 삼성전자 인덕션 2구 제외 추천
LLM Query: 삼성전자 인덕션 제품 중에서 2구 모델을 제외한, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 모델을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 설명해주시면 좋겠습니다.
Web Search Results: 3


삼성전자는 다양한 인덕션 제품을 제공하며, 3구 이상의 모델도 여러 가지가 있습니다. 아래는 추천할 만한 몇 가지 모델과 그 주요 기능, 장점입니다. 🍳✨

### 1. **Bespoke AI 인덕션 (플렉스존)**
- **화구 수**: 4구
- **주요 기능**:
  - **플렉스존**: 화구를 자유롭게 조절할 수 있어 큰 팬이나 냄비를 동시에 사용할 수 있습니다.
  - **AI 화력 조절**: 요리하는 재료에 맞춰 자동으로 최적의 화력을 조절합니다.
- **장점**:
  - 다채로운 요리 가능: 여러 가지 요리를 동시에 할 수 있어 효율적입니다.
  - 사용자 맞춤형 조리: AI가 요리에 맞는 화력을 설정해 주기 때문에 간편하게 요리할 수 있습니다.

[더 알아보기](https://www.samsung.com/sec/cooking-appliances/)

### 2. **더 플레이트 인덕션**
- **화구 수**: 3구
- **주요 기능**:
  - **각각 독립적으로 사용 가능**: 각 화구의 온도와 설정을 독립적으로 조절할 수 있습니다.
  - **고화력 지원**: 최대 3.3 kW의 고화력을 지원하여 빠른 조리가 가능합니다.
- **장점**:
  - 조리 공간을 최적화: 동시에 여러 요리를 할 수 있어 주방 공간 활용이 뛰어납니다.
  - 세련된 디자인: 화이트와 블랙의 슬림 디자인으로 모던한 주방 인테리어에 잘 어울립니다.

[자세히 보기](https://www.samsung.com/sec/electric-range/cooktop-nz62dg300cfk-d2c/NZ62DG300CFK/)

### 3. **삼성전자 전자레인지 인덕션 콤비 모델**
- **화구 수**: 3구 (인덕션) + 전자레인지 기능
- **주요 기능**:
  - **전자레인지와 인덕션의 콤비네이션**: 인덕션 요리뿐만 아니라 전자레인지 기능도 제공하여 다용도로 활용 가능합니다.
  - **스마트 기능**: SmartThings와 연결하여 요리를 쉽게 제어할 수 있습니다.
- **장점**:
  - 복합적인 요리 가능: 전자레인지 기능을 추가하여 더 많은 요리를 할 수 있어 요리의 효율성을 높입니다.
  - 공간 절약: 하나의 기기로 다양한 기능을 수행할 수 있어 주방 공간 활용이 좋습니다.

[제품 정보 확인하기](https://www.samsung.com/sec/cooking-appliances/)

이 모델들은 각각의 특징과 장점이 있어 사용자의 필요에 따라 선택할 수 있습니다. 추가적인 정보가 필요하시거나 다른 모델에 대한 질문이 있다면 언제든지 말씀해 주세요! 😊

elapsed time: 10.59 seconds
Bing Grounding 검색 사용: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Web Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.
Existing agent ID: asst_1BNu5p4Wv52Cload5HkW3ZBa
Web Search Results: 4


삼성전자의 3구 이상의 인덕션 모델을 추천해 드리겠습니다! 😊🔥 아래는 인기 있는 3가지 모델과 그 주요 기능 및 장점입니다.

1. **BESPOKE AI Induction (NZ63DB657CFE)**
   - **주요 기능**: 3개의 버너를 동시에 사용 가능, 자동 전력 조정 기능, MAX Boost로 고열 조리 가능.
   - **장점**: 효율적인 전력 관리와 강력한 열출력으로 빠른 요리가 가능해요. 
   - [자세히 보기](https://www.samsung.com/sec/electric-range/cooktop-nz63db657cfe-d2c/NZ63DB657CFE/)

2. **Induction (NZ63B5056AK)**
   - **주요 기능**: 대형 듀얼 링 버너, Quattro Flex Zone으로 다양한 조리 가능, 최대 3,400W 출력.
   - **장점**: 대형 조리기구를 수용할 수 있으며, 강력한 열 공급으로 다양한 요리가 용이합니다.
   - [자세히 보기](https://www.samsung.com/sec/electric-range/cooktop-nz63b5056cb-d2c/NZ63B5056AK/)

3. **BESPOKE AI Induction Infinite Line (NZ63DB857CAV)**
   - **주요 기능**: 3개의 버너, MAX Boost 기능, 자동 전력 조정.
   - **장점**: 프리미엄 디자인과 높은 성능으로 까다로운 조리 요구를 만족시킵니다.
   - [자세히 보기](https://www.samsung.com/sec/electric-range/cooktop-nz63db857cav-d2c/NZ63DB857CAV/)

이 모델들은 모두 삼성전자의 최신 기술을 적용하여 효율적이고 편리한 요리를 지원합니다. 필요에 따라 적합한 모델을 선택해 보세요! 😊

elapsed time: 18.79 seconds


In [15]:

RESULTS_COUNT = 10

inputs = [
    #"삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    # "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    # "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    # "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]


WEB_SEARCH_MODE = "google"  

for input in inputs:
    
    print(f"Google Search API 사용: {input}")
    await process_web_search_call(RESULTS_COUNT, input)

WEB_SEARCH_MODE = "bing"

for input in inputs:
    print(f"Bing Grounding 검색 사용: {input}")
    await process_web_search_call(RESULTS_COUNT, input)    

Google Search API 사용: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘


Web Search Query: 삼성전자 TV 추천
LLM Query: 부모님에게 선물할 삼성전자 TV 모델을 추천해 주세요. 추천 모델의 주요 기능과 장점도 함께 설명해 주시면 좋겠습니다.
Web Search Results: 10


부모님께 선물할 삼성전자 TV 모델로 **삼성 QLED 4K TV**를 추천드립니다! 🎁

### 삼성 QLED 4K TV의 주요 기능과 장점

1. **우수한 화질**: QLED 기술이 적용되어 생생하고 선명한 색감을 제공합니다. 특히, HDR(High Dynamic Range) 기능으로 더 넓은 색 영역과 깊이 있는 명암을 구현하여 실감나는 영상 경험을 선사합니다.

2. **AI 업스케일링**: Neo 퀀텀 프로세서가 탑재되어 있어, 저해상도 콘텐츠도 4K 화질로 업스케일링해줍니다. 이를 통해 부모님께서 과거의 영상도 더욱 선명하게 즐길 수 있습니다. 이 기술은 최신 AI 기술이 적용되어 더욱 향상된 화질을 제공합니다. 🌈

3. **스마트 기능**: 다양한 스트리밍 서비스와 호환되어 Netflix, YouTube 등 인기 콘텐츠를 쉽게 이용할 수 있습니다. 또한, 스마트 허브 기능으로 여러 앱을 한 곳에서 관리할 수 있어 사용이 간편합니다.

4. **편리한 설치 및 디자인**: 슬림한 디자인과 다양한 설치 옵션으로 부모님의 집안 인테리어와 잘 어울립니다. 벽걸이 설치도 가능하여 공간을 절약할 수 있습니다.

5. **소리 품질**: Dolby Atmos 지원으로 몰입감 있는 사운드를 제공하여 영화 관람 시 더욱 실감나는 경험을 선사합니다.

부모님께서 TV를 통해 다양한 콘텐츠를 즐기실 수 있도록 이러한 기능들이 잘 갖춰진 삼성 QLED 4K TV를 고려해보시는 것이 좋습니다. 추가적인 정보나 구매는 [삼성 공식 웹사이트](https://www.samsung.com/sec/tvs/)에서 확인해보세요! 🖥️

elapsed time: 7.84 seconds
Google Search API 사용: 삼성전자 25년 제품이 작년 대비 좋아진것은
Original Input: 삼성전자 25년 제품이 작년 대비 좋아진것은
Web Search Query: 삼성전자 2025년 제품 작년 대비 개선 사항
LLM Query: 삼성전자의 2025년 제품이 작년 대비 어떤 점에서 개선되었는지 구체적으로 알려주세요. 특히 기술적 혁신이나 성능 향상에 대해 설명해주시면 감사하겠습니다.
Web Search Results: 10


삼성전자는 2025년 제품에서 여러 가지 기술적 혁신과 성능 향상을 이루었습니다. 주요 개선 사항은 다음과 같습니다:

### 1. 고대역폭 메모리(HBM) 기술 경쟁력 강화 🧠
삼성전자는 HBM과 같은 첨단 반도체 기술의 경쟁력을 더욱 강화하고 있습니다. 특히, HBM3E와 같은 차세대 GPU에 맞춤화된 제품 개발이 진행 중이며, 이는 높은 데이터 전송 속도를 필요로 하는 AI 및 머신러닝 응용에 최적화되어 있습니다. 이러한 기술 발전은 삼성전자의 메모리 사업에 긍정적인 영향을 미칠 것으로 기대됩니다. [더 알아보기](https://blog.naver.com/arcrealty/223719733564)

### 2. 파운드리 기술 및 AI 솔루션 통합
삼성전자는 AI 기술과 결합된 최첨단 파운드리 기술을 제공하고 있습니다. "Empowering the AI Revolution"을 주제로 한 파운드리 포럼에서 삼성전자는 GAA(Gate-All-Around) 공정 기술을 통해 저전력 소모로 고속 데이터 처리가 가능한 반도체 솔루션을 소개했습니다. 이는 고객들에게 AI 솔루션을 구현하는 데 필요한 강력한 지원을 제공합니다. [자세히 보기](https://www.samsung.com/sec/business/insights/news/news_240617_03/)

### 3. 생산 공정 개선 및 성능 향상
삼성전자는 1.4나노 공정의 양산을 계획하고 있으며, 기존의 나노 공정 대비 광학적 축소(optical shrink)를 통해 전력 소비(PPA)를 추가로 향상시킬 수 있도록 준비하고 있습니다. 이러한 기술적 혁신은 반도체의 성능을 극대화하며, 다양한 응용 분야에 적합한 솔루션을 제공합니다. [더 알아보기](https://grimstock.tistory.com/entry/%EB%85%B9%EC%8B%AD%EC%9E%90%EC%9B%B0%EB%B9%99-2025%EB%85%84-1%EB%B6%84%EA%B8%B0-%EC%8B%A4%EC%A0%81%EA%B3%B5%EC%8B%9C-%EC%9A%94%EC%95%BD-%EB%B0%8F-%EC%B0%B8%EA%B3%A0%EC%82%AC%ED%95%AD)

### 4. 영업이익 및 수익성 개선
2025년 1분기 실적에서 삼성전자는 영업이익이 전년 대비 81.6% 증가하고, 당기순이익은 391.3% 증가하는 성과를 거두었습니다. 이는 고수익 제품의 판매 증가와 비용 효율화 전략의 결과로, 제품의 기술적 혁신이 실제 성과로 이어졌음을 보여줍니다. [자세한 실적 정보](https://grimstock.tistory.com/entry/%EB%85%B9%EC%8B%AD%EC%9E%90%EC%9B%B0%EB%B9%99-2025%EB%85%84-1%EB%B6%84%EA%B8%B0-%EC%8B%A4%EC%A0%81%EA%B3%B5%EC%8B%9C-%EC%9A%94%EC%95%BD-%EB%B0%8F-%EC%B0%B8%EA%B3%A0%EC%82%AC%ED%95%AD)

이러한 여러 개선 사항들은 삼성전자가 기술적 리더십을 유지하며, 시장의 요구에 적극적으로 대응하고 있음을 보여줍니다. 🚀

elapsed time: 12.86 seconds
Bing Grounding 검색 사용: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Web Search Query: 삼성전자 TV 추천 부모님 선물용
LLM Query: 부모님에게 선물할 삼성전자 TV 모델을 추천해 주세요. 다양한 가격대와 기능을 고려하여 최적의 선택을 할 수 있도록 각 모델의 장점과 특징도 함께 설명해 주세요.
Existing agent ID: asst_1BNu5p4Wv52Cload5HkW3ZBa
Web Search Results: 11


부모님에게 선물할 삼성전자 TV 모델을 추천해드릴게요! 여러 가격대와 기능을 고려하여 최적의 선택을 할 수 있도록 각 모델의 장점과 특징을 아래에 정리해 보았습니다. 🎁📺

### 1. **Samsung QN90A (Neo QLED)**
- **가격대**: 중고가
- **장점**: 
  - 뛰어난 화질: Quantum HDR와 4K 해상도로 생생한 화면 제공.
  - 뛰어난 명암비와 색재현력: Neo QLED 기술을 적용해 깊은 검은색과 선명한 색감을 표현.
  - 스마트 기능: Tizen OS 기반으로 다양한 앱과 콘텐츠에 쉽게 접근 가능.

### 2. **Samsung AU8000 (Crystal UHD)**
- **가격대**: 저가
- **장점**: 
  - 경제적인 가격: 기능 대비 합리적인 가격으로 가성비 좋음.
  - 선명한 화면: Crystal UHD 기술로 좋은 화질 제공.
  - 사용 편리함: Smart Hub 기능으로 다양한 콘텐츠를 쉽게 탐색 가능.

### 3. **Samsung Q80A (QLED)**
- **가격대**: 중고가
- **장점**: 
  - 뛰어난 게임 성능: VRR(가변 주사율)과 ALLM(자동 저지연 모드) 지원으로 게임용으로도 최적.
  - 풍부한 사운드: OTS(Objects Tracking Sound) 기술로 몰입감 있는 음향 제공.
  - 다양한 크기 옵션: 부모님 방 크기에 맞춰 선택 가능.

### 4. **Samsung Frame (2022)**
- **가격대**: 고가
- **장점**: 
  - 예술 작품처럼 장식 가능: TV를 껐을 때도 예술 작품으로 사용할 수 있어 인테리어 효과.
  - 개인 맞춤형: 다양한 아트 모드와 프레임 옵션으로 디자인 맞춤화 가능.
  - 4K 화질: 고화질 콘텐츠 감상 가능.

이 외에도 더 많은 모델과 자세한 정보를 확인하고 싶다면 [삼성전자 공식 웹사이트](https://www.samsung.com/sec/tvs/)를 방문해 보세요. 추가로 각 모델의 비교 및 추천에 대한 더 많은 정보를 원하신다면 아래 링크를 참고하실 수 있습니다:

- [삼성전자 TOP 4 리뷰 & 비교 (2025년)](https://nosearch.com/recommendation/pick/tv/tv)
- [2025년형 TV 40종 비교](https://monna2.tistory.com/531)

부모님께서 사용하실 TV는 기호에 따라 다를 수 있으니, 기능과 디자인을 고려하여 최적의 선택을 하시길 바랍니다! 🌟

elapsed time: 19.93 seconds
Bing Grounding 검색 사용: 삼성전자 25년 제품이 작년 대비 좋아진것은
Original Input: 삼성전자 25년 제품이 작년 대비 좋아진것은
Web Search Query: 삼성전자 2025년 제품 작년 대비 개선점
LLM Query: 삼성전자의 2025년 제품은 작년 모델과 비교하여 어떤 점에서 개선되었는지 알려주세요. 특히 주요 기능, 성능, 디자인 변화에 대해 설명해주시면 좋겠습니다.
Existing agent ID: asst_1BNu5p4Wv52Cload5HkW3ZBa
Web Search Results: 11


삼성전자의 2025년 제품은 작년 모델에 비해 여러 면에서 눈에 띄는 개선이 이루어졌습니다. 여기서 주요 기능, 성능, 디자인 변화에 대해 자세히 설명드리겠습니다. 🚀✨

### 1. 주요 기능 개선
- **AI 기술**: 2025년 모델에서는 AI 기술이 크게 향상되어, 특히 TV 제품군에서 진정한 AI TV 시대를 선언하였습니다. 이는 자동 화면 조정, 음성 인식, 사용자 맞춤형 콘텐츠 추천 등을 포함합니다. [삼성전자 2025년형 TV 신제품](https://news.samsung.com/kr/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-2025%EB%85%84%ED%98%95-tv-%EC%8B%A0%EC%A0%9C%ED%92%88-%EB%9D%BC%EC%9D%B8%EC%97%85-%EA%B3%B5%EA%B0%9C-%EC%A7%84%EC%A0%95%ED%95%9C-ai-tv-%EC%8B%9C%EB%8C%80-2)

### 2. 성능 향상
- **스마트폰 성능**: Galaxy S25는 전작보다 성능이 약 40% 향상되었습니다. 특히, 프로세서와 GPU의 성능이 개선되어 보다 빠른 처리 속도를 제공합니다. [갤S25 성능](https://www.asiatoday.co.kr/kn/view.php?key=20250122010012190)

### 3. 디자인 변화
- **디자인 혁신**: 2025년 모델들은 새로운 디자인 언어를 채택하여 둥근 모서리와 세련된 라인을 강조합니다. 이러한 변화는 제품의 사용성을 높이고, 더 나아가 시각적으로도 매력적입니다. [디자인 어워드 수상](https://news.samsung.com/kr/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-if-%EB%94%94%EC%9E%90%EC%9D%B8-%EC%96%B4%EC%9B%8C%EB%93%9C-2025%EC%97%90%EC%84%9C-%EA%B8%88%EC%83%81-2%EA%B0%9C-%ED%8F%AC%ED%95%A8-%EC%B4%9D-58%EA%B0%9C)

### 4. 제품 라인업
- **갤럭시 A 시리즈**: 2025년에는 Galaxy A56 5G와 A36 5G 등 새로운 보급형 모델이 출시되었습니다. 이 모델들은 뛰어난 가격 대비 성능을 제공하며, 다양한 사용자층을 겨냥하고 있습니다. [갤럭시 A 시리즈 비교](https://blog.naver.com/artismart/223870094344)

이 외에도 삼성전자는 2025년 제품에 다양한 혁신을 도입하여 소비자들에게 더 나은 경험을 제공하고 있습니다. 추가적인 정보나 구체적인 모델에 대한 궁금증이 있으시면 언제든지 물어보세요! 😊

elapsed time: 24.17 seconds
